## PHASE 4 PROJECT: **Building a Time Series Model using Zillow Housing Dataset**

## OVERVIEW

Real estate industry is a highly capital intensive venture and thus extensive, accurate and predictive analysis is key. In this project I will conduct time series data analysis and gather valuable insights which will aid a mid-income house buyer make a house purchase decision wisely. The scope of this project ranges from Overall Business Understanding, Setting up Objectives, Data Understanding, Data Analysis, Data Modeling as well as conclusion and Recommendations. 

## 1. Business Understanding

Real estate sector is very dynamic and a major industry in all economies of the world. Most Americans dream of owning a house and thus Real estate reports and Analysis are vital in keeping them informed. This project utilizes historical data to paint a picture of major trends in the real estate sector over years and form basis of predicting future house prices. Key market factors that will aid in the modeling process will be the location of the houses as per the zip codes. 

## 2. Objectives

The overall objective of this project is to identify the best localities to invest in based on the house prices. 
Analyzing historical data to identify house price trends across different zip codes


## 3. Data Understanding

The dataset to be used in this project is 'zillow_data.csv' sourced from [Zillow Housing Data](https://www.zillow.com/research/data/). Key columns in this dataset include: RegionID, RegionName, City, State, Metro, SizeRank, CountyName, and value (real estate prices). 

#### Import Necessary Libraries

In [1]:

#Importing data libraries
import numpy as np
import pandas as pd

#importing visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#importing math libraries
from math import sqrt

#Importing modeling libraries
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib.pylab import rcParams
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error as MSE
     

Pre-viewing the Dataframe

In [2]:
# Opening the dataframe and pre-viewing it
df= pd.read_csv('zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [3]:
#Checking the shape of the dataset
df.shape   

(14723, 272)

## 4. Data Wrangling

##### Checking for missing values

In [4]:
df.isna().sum()

RegionID         0
RegionName       0
City             0
State            0
Metro         1043
              ... 
2017-12          0
2018-01          0
2018-02          0
2018-03          0
2018-04          0
Length: 272, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [6]:
# For clarity we have to display all the rows with the missing data
df[df.isnull().any(axis=1)].iloc[:-1]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
20,61625,10011,New York,NY,New York,New York,21,NaN,NaN,NaN,...,12137600,12112600,12036600,12050100,12016300,11946500,11978100,11849300,11563000,11478300
36,61796,10456,New York,NY,New York,Bronx,37,NaN,NaN,NaN,...,357900,357100,356500,357200,362000,368500,374100,379800,388100,394400
105,84613,60611,Chicago,IL,Chicago,Cook,106,NaN,NaN,NaN,...,1475200,1473900,1469500,1472100,1477800,1486000,1497300,1508600,1508100,1493200
151,69340,27410,Greensboro,NC,NaN,Guilford,152,137100.0,136600.0,136000.0,...,212900,213200,213600,214300,215100,216200,217700,219600,221000,221500
156,62048,11238,New York,NY,New York,Kings,157,NaN,NaN,NaN,...,2673300,2696700,2716500,2724000,2744300,2768900,2810700,2848100,2869600,2885300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14706,59046,3215,Waterville Valley,NH,Claremont,Grafton,14707,NaN,NaN,NaN,...,786000,780900,774100,767800,778300,796800,808100,811600,806500,796600
14707,69681,28039,East Spencer,NC,Charlotte,Rowan,14708,NaN,NaN,NaN,...,27300,26400,25500,25100,25100,25600,25800,26600,27700,28300
14708,99401,97733,Crescent,OR,Klamath Falls,Klamath,14709,NaN,NaN,NaN,...,197700,203700,207900,208100,206400,208700,211800,218100,228400,238500
14710,59210,3812,Bartlett,NH,NaN,Carroll,14711,80900.0,80800.0,80800.0,...,215500,217000,219200,221700,223600,224800,226000,226900,227700,228000


#### Dealing with missing Data

The ideal method to use when dealing with missing values in a time series analysis is linear interpolation method. However, this technique will be limited to cleaning numerical data only.  

In [7]:
# First I will look for missing values in each column
missing_columns = df.columns[df.isnull().any()].tolist()
print(missing_columns)

['Metro', '1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09', '1996-10', '1996-11', '1996-12', '1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06', '1997-07', '1997-08', '1997-09', '1997-10', '1997-11', '1997-12', '1998-01', '1998-02', '1998-03', '1998-04', '1998-05', '1998-06', '1998-07', '1998-08', '1998-09', '1998-10', '1998-11', '1998-12', '1999-01', '1999-02', '1999-03', '1999-04', '1999-05', '1999-06', '1999-07', '1999-08', '1999-09', '1999-10', '1999-11', '1999-12', '2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10', '2000-11', '2000-12', '2001-01', '2001-02', '2001-03', '2001-04', '2001-05', '2001-06', '2001-07', '2001-08', '2001-09', '2001-10', '2001-11', '2001-12', '2002-01', '2002-02', '2002-03', '2002-04', '2002-05', '2002-06', '2002-07', '2002-08', '2002-09', '2002-10', '2002-11', '2002-12', '2003-01', '2003-02', '2003-03', '2003-04', '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', 

In [8]:
# Iterate through columns with missing values
for col in missing_columns:
    # interpolate using linear method
    df[col] = df[col].interpolate(method='linear')

# Print the updated dataset with interpolated values
print(df)

       RegionID  RegionName                 City State              Metro  \
0         84654       60657              Chicago    IL            Chicago   
1         90668       75070             McKinney    TX  Dallas-Fort Worth   
2         91982       77494                 Katy    TX            Houston   
3         84616       60614              Chicago    IL            Chicago   
4         93144       79936              El Paso    TX            El Paso   
...         ...         ...                  ...   ...                ...   
14718     58333        1338             Ashfield    MA    Greenfield Town   
14719     59107        3293            Woodstock    NH          Claremont   
14720     75672       40404                Berea    KY           Richmond   
14721     93733       81225  Mount Crested Butte    CO                NaN   
14722     95851       89155             Mesquite    NV          Las Vegas   

      CountyName  SizeRank   1996-04   1996-05   1996-06  ...  2017-07  \
0

The other categorical variable column with missing values is 'Metro'. It is not easy to come up with a way of filling them up, so the only way is to drop the null values.

In [9]:
df = df.drop('Metro', axis = 1)

In [10]:

#Confirming existence of missing values
df[df.isnull().any(axis=1)].iloc[:-1]

,RegionID,RegionName,City,State,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04


##### Reshaping the dataset

In [11]:
# Changing dataframe time column from float to datetime format
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects
    as datetime objects.
    number of returned columns may not match total number of columns in passed dataframe
    """
    return pd.to_datetime(df_new.columns.values[1:], format='%Y-%m')

Next we will manipulate our data from wide to long format. This will help to minimize the number of columns and ease the analysis.

In [12]:
def melt_data(data):
    melted = pd.melt(data, id_vars=['RegionID', 'RegionName', 'City', 'State', 'SizeRank', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [21]:
# Previewinga few rows of the long format dataframe
long_df = melt_data(df)
long_df.head()

,RegionID,RegionName,City,State,SizeRank,CountyName,time,value
0,84654,60657,Chicago,IL,1,Cook,1996-04-01,334200.0
1,90668,75070,McKinney,TX,2,Collin,1996-04-01,235700.0
2,91982,77494,Katy,TX,3,Harris,1996-04-01,210400.0
3,84616,60614,Chicago,IL,4,Cook,1996-04-01,498100.0
4,93144,79936,El Paso,TX,5,El Paso,1996-04-01,77300.0


In [22]:
long_df.describe()

,RegionID,RegionName,SizeRank,value
count,3.901595e+06,3.901595e+06,3.901595e+06,3.901595e+06
mean,8.107501e+04,4.822235e+04,7.362000e+03,2.060636e+05
std,3.193304e+04,2.935833e+04,4.250165e+03,2.368017e+05
min,5.819600e+04,1.001000e+03,1.000000e+00,1.130000e+04
25%,6.717400e+04,2.210100e+04,3.681000e+03,9.770000e+04
50%,7.800700e+04,4.610600e+04,7.362000e+03,1.469000e+05
75%,9.092100e+04,7.520600e+04,1.104300e+04,2.354000e+05
max,7.538440e+05,9.990100e+04,1.472300e+04,1.931490e+07


In [23]:
long_df.isna().sum()

RegionID      0
RegionName    0
City          0
State         0
SizeRank      0
CountyName    0
time          0
value         0
dtype: int64

In [24]:
long_df.duplicated().sum()

0

## 5. Explanatory Data Analysis

We begin by setting time as the index of the new dataframe

In [25]:
#set time as the index
long_df.set_index('time', inplace = True)

We resample the the dataframe based on the months with time as the index

In [26]:
#Resampling dataframe using months
monthly = long_df.resample('MS')
month_mean = monthly.mean()
month_mean.tail()

,RegionID,RegionName,SizeRank,value
time,,,,
2017-12-01,81075.010052,48222.348706,7362.0,281095.320247
2018-01-01,81075.010052,48222.348706,7362.0,282657.060382
2018-02-01,81075.010052,48222.348706,7362.0,284368.688447
2018-03-01,81075.010052,48222.348706,7362.0,286511.376757
2018-04-01,81075.010052,48222.348706,7362.0,288039.944305
